In [1]:
from __init__ import *
from config import *
from inference import *
from main import *
from model import *
from training import *
from utils import *

In [2]:
import datetime
import logging
import os

def run_yolo_inference():
    # Set default values
    debug = False
    verbose = False
    log_dir = 'logs'
    cpu_only = False
    out_dir = 'output'
    weight_path = 'yolov3_original.pt'
    from_ckpt = False
    img_dir = './data/samples'
    img_size = 416
    batch_size = 8
    n_cpu = 4
    conf_thres = 0.5
    nms_thres = 0.4
    save_det = True
    save_img = True
    class_path = 'coco.names'

    # configure logging
    current_datetime_str = datetime.datetime.now().strftime("%Y-%m-%d-%H-%M-%S")
    log_file_name_by_time = current_datetime_str + ".log"
    if debug:
        log_level = logging.DEBUG
    elif verbose:
        log_level = logging.INFO
    else:
        log_level = logging.WARNING
    config_logging(log_dir, log_file_name_by_time, level=log_level)

    # set the device for inference
    dev = config_device(cpu_only)
    make_output_dir(out_dir)

    # load model
    model = load_yolov3_model(weight_path, dev, ckpt=from_ckpt)

    # load data
    dataloader = load_dataset(type='image_folder',
                              img_dir=img_dir,
                              annot_dir=None,
                              img_size=img_size,
                              batch_size=batch_size,
                              n_cpu=n_cpu,
                              shuffle=False,
                              augment=False)

    # run detection
    results = run_detection(model, dataloader, dev, conf_thres, nms_thres)

    # post processing
    if save_det:
        json_path = '{}/{}/detections.json'.format(out_dir, current_datetime_str)
        make_output_dir(os.path.split(json_path)[0])
        save_results_as_json(results, json_path)

    if save_img:
        class_names = load_classes(class_path)
        img_path = '{}/{}/img'.format(out_dir, current_datetime_str)
        make_output_dir(img_path)
        save_results_as_images(results, img_path, class_names)

    return


In [3]:
run_yolo_inference()